In [16]:
from IPython.display import display, Markdown, Latex

In [46]:
from __future__ import annotations
##
import asyncio
lock_tt = asyncio.Lock()
##
DAY_START = 5
##
from peewee import *
from brish import z
import os
from dateutil.relativedelta import relativedelta
from pathlib import Path

# Path.home().joinpath(Path("cellar"))
db_path = Path(
    z('print -r -- "${{attic_private_dir:-$HOME/tmp}}/timetracker.db"').outrs)
os.makedirs(os.path.dirname(db_path), exist_ok=True)
db = SqliteDatabase(db_path)


class BaseModel(Model):
    class Meta:
        database = db


class Activity(BaseModel):
    name = CharField()
    start = DateTimeField()
    end = DateTimeField()

    def __str__(self):
        dur = relativedelta(self.end, self.start)
        return f"""{self.name} {relativedelta_str(dur)}"""

## indexes: add manually via Datagrip (right-click on table, modify table)(adding it via peewee is not necesseray https://github.com/coleifer/peewee/issues/2360 )
# create index activity_end_index
#     on activity (end desc);
# create index activity_start_end_index
#     on activity (start desc, end desc);
##

db.close()
db.connect()  # @todo? db.close()
db.create_tables([Activity])
##

import textwrap
import dataclasses
from dataclasses import dataclass
from functools import total_ordering
from typing import Dict, List
import datetime
from dateutil.relativedelta import relativedelta


def relativedelta_total_seconds(rd: relativedelta):
    # Used Google to convert the years and months, they are slightly more than 365 and 30 days respectively.
    return rd.years * 31540000 + rd.months * 2628000 + rd.days * 86400 + rd.hours * 3600 + rd.minutes * 60 + rd.seconds


def gen_s(num):
    if num != 1:
        return "s"
    return ""


def relativedelta_str(rd: relativedelta):
    res = ""
    rd = rd.normalized()
    # rd.weeks seems to just convert rd.days into weeks
    if rd.years:
        res += f"{rd.years} year{gen_s(rd.years)}, "
    if rd.months:
        res += f"{rd.months} month{gen_s(rd.months)}, "
    if rd.days:
        res += f"{rd.days} day{gen_s(rd.days)}, "
    if rd.hours:
        res += f"{rd.hours}:"
    res += f"{rd.minutes}"
    return res

@total_ordering
@dataclass()
class ActivityDuration:
    # @legacyComment Somehow putting ActivityDuration in the plugin file itself resulted in error (the culprit was probably dataclass), so I am putting them here.
    name: str
    duration: relativedelta = dataclasses.field(default_factory=relativedelta)
    sub_acts: Dict[str, ActivityDuration] = dataclasses.field(
        default_factory=dict)

    total_duration: relativedelta = dataclasses.field(
        default_factory=relativedelta)
    # @property
    # def total_duration(self):
    #     res = self.duration
    #     for act in self.sub_acts:
    #         res += act.total_duration
    #     return res

    def __lt__(self, other):
        if type(other) is ActivityDuration:
            return relativedelta_total_seconds(self.total_duration) < relativedelta_total_seconds(other.total_duration)
        elif type(other) is relativedelta:
            return relativedelta_total_seconds(self.total_duration) < relativedelta_total_seconds(other)
        else:
            return NotImplemented

    def add(self, dur: relativedelta, act_chain: List[str]):
        self.total_duration += dur
        if len(act_chain) == 0:
            self.duration += dur
        else:
            # act_chain's last item should be the parent for possible perf reasons
            child = act_chain.pop()
            child_act = self.sub_acts.setdefault(
                child, ActivityDuration(name=child))
            child_act.add(dur, act_chain)

    def __str__(self, width=25, indent="  "):
        def adjust_name(name, width=width):
            return name + " " * max(4, width - len(name))

        res = ""
        name = self.name
        skip_me = (name == "Total")  # Skip root
        my_indent = indent
        next_width = width - len(indent)
        if not skip_me:
            res += f"""{adjust_name(name)} {relativedelta_str(self.total_duration)}\n"""
            if len(self.sub_acts) > 0 and relativedelta_total_seconds(self.duration) > 60:
                res += f"""{my_indent}{adjust_name(".", width=next_width)} {relativedelta_str(self.duration)}\n"""
        else:
            my_indent = ""
        for act in sorted(self.sub_acts.values(), reverse=True):
            res += textwrap.indent(act.__str__(width=(next_width), indent=indent), my_indent)
        return res

##
def activity_list_to_str_now(delta=datetime.timedelta(hours=24)):
    now = datetime.datetime.today()
    low = now - delta
    return activity_list_to_str(low,now)

def activity_list_to_str(low, high):
    acts = Activity.select().where((Activity.start.between(low, high)) | (Activity.end.between(low, high)))
    global acts_agg
    acts_agg = ActivityDuration("Total")
    for act in acts:
        act_name = act.name
        act_start = max(act.start, low)
        act_end = min(act.end, high)
        dur = relativedelta(act_end, act_start)
        acts_agg.add(dur, list(reversed(act_name.split('_'))))
    # ("TOTAL", total_dur),
    # we need a monospace font to justify the columns
    res = f"```\nSpanning {str(high - low)}; UNACCOUNTED {relativedelta_str(relativedelta(high, low + acts_agg.total_duration))}\n"
    res += str(acts_agg)
    return res + "\n```"


def activity_list_habit_get_now(name: str, delta=datetime.timedelta(days=30), mode=0, fill_default=True):
    # _now means 'now' is 'high'
    high = datetime.datetime.today()
    low = high - delta
    low = low.replace(hour=DAY_START, minute=0, second=0, microsecond=0)
    # aligns dates with real life, so that date changes happen at, e.g., 5 AM
    night_passover = datetime.timedelta(hours=DAY_START)

    def which_bucket(act: Activity):
        if act.name == name or act.name.startswith(name + '_'):
            return (act.start - night_passover).date()
        return None

    buckets = activity_list_buckets_get(
        low, high, which_bucket=which_bucket, mode=mode)
    if mode == 0:
        buckets_dur = {k: round(relativedelta_total_seconds(
            v.total_duration) / 3600, 2) for k, v in buckets.items()}
    elif mode == 1:
        buckets_dur = buckets

    if fill_default:
        interval = datetime.timedelta(days=1)
        while low <= high:
            buckets_dur.setdefault(low.date(), 0)
            low += interval

    return buckets_dur


def activity_list_buckets_get(low, high, which_bucket, mode=0, correct_overlap=True):
    acts = None
    # adding the name query here will increase performance. (Currently done in which_bucket.)
    if correct_overlap:
        acts = Activity.select().where((Activity.start.between(low, high)) | (Activity.end.between(low, high)))
    else:
        acts = Activity.select().where((Activity.start.between(low, high)))

    buckets = {}
    for act in acts:
        if correct_overlap:
            # [Q] Is it possible to mark a model object as "unsave-able"? https://github.com/coleifer/peewee/issues/2375
            act.start = max(act.start, low)
            act.end = min(act.end, high)

        bucket_key = which_bucket(act)
        if not bucket_key:
            continue
        if mode == 0:
            bucket = buckets.setdefault(bucket_key, ActivityDuration("Total"))
            dur = relativedelta(act.end, act.start)
            bucket.add(dur, list(reversed(act.name.split('_'))))
        elif mode == 1:  # count mode
            bucket = buckets.setdefault(bucket_key, 0)
            buckets[bucket_key] += 1
    return buckets


In [116]:
def get_acts(root: ActivityDuration):
    # mutates its input! is not idempotent!
    acts = [root]
    if not hasattr(root, 'parent'):
        root.parent = None
    if not hasattr(root, 'shortname'):
        root.shortname = root.name

    act: ActivityDuration
    for act in root.sub_acts.values():
        act.parent = root
        act.shortname = act.name
        if act.parent.name != 'Total':
            act.name = f"{act.parent.name}_{act.name}"
        # print(f"{act.shortname} -> {act.name} via parent {act.parent.name}")
        acts += get_acts(act)
    return acts

db.close()
db.connect()
a = activity_list_to_str_now(delta=datetime.timedelta(days=7)) # reset acts_agg
print(a)
all_acts = get_acts(acts_agg)
len(all_acts)


```
Spanning 7 days, 0:00:00; UNACCOUNTED 1:15
sleep                   2 days, 6:8
sa                      2 days, 4:21
  .                     1 day, 9:21
  development           17:30
    .                   7:45
    shiny               7:5
    quantified self     2:40
      timetracker       2:40
  p4v                   1:17
  product               10
    evaluation          10
study                   1 day, 7:28
  physics               15:30
    physics 4           9:41
      .                 7:35
      video             1:20
      hw                46
    physics 2           5:49
      .                 3:58
      quizz             1:33
      debug             16
  cs                    8:0
    ai                  8:0
      .                 5:55
      hw                1:37
      halfhearted       28
  math                  5:38
    probability and statistics     5:38
      video             5:38
        .               3:26
        eat             1:6
        lunch           46

89

In [138]:
# print(acts_agg)
import plotly.graph_objects as go

ids = None
labels = [act.name for act in all_acts]
texts = [act.shortname for act in all_acts]
## Make displayed labels short (recommended):
ids = labels
labels = texts
texts = ids
##
parents = [(act.parent and act.parent.name) or "" for act in all_acts]
# values = [relativedelta_total_seconds(act.duration) for act in all_acts]
values = [relativedelta_total_seconds(act.total_duration)/(3600) for act in all_acts]

# lim=19
# labels = labels[:lim]
# parents = parents[:lim]
# values = values[:lim]

# print(labels)
# print(texts)
# print(parents)
# print(values)

# https://plotly.com/python/treemaps/
# https://plotly.com/python/reference/treemap/
fig = go.Figure(go.Treemap(
    branchvalues = "total",
    ids = ids,
    labels = labels,
    parents = parents,
    values = values,
    text = texts,
    # textinfo = "text+value+percent parent+percent entry+percent root",
    textinfo = "label+value+percent parent+percent entry+percent root",
    hoverinfo = "label+text+value+percent parent+percent entry+percent root",
    # hoverinfo = "label+value+percent parent+percent entry+percent root",
))
# fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))
fig.update_layout(margin = dict(t=30, l=0, r=30, b=30))
# fig.update_layout(uniformtext=dict(minsize=6, mode='hide'))
fig.show()

# fig = go.Figure(go.Treemap(
#     # branchvalues = "total",
#     labels = labels,
#     parents = parents,
#     values = values
# ))

# fig.show()